# I. US County Race Population

In [1]:
import pandas as pd
import numpy as np 
from datetime import datetime  


def diagnose_Rate(T, **kwargs):
    info_list = []
    for col in T.columns:
        if col in ['Rate_Black', 'Rate_White', 'Rate_Parity']:
            d = {'name': col}
            for k, v in kwargs.items():
                d[k] = v
                
            for k,v in T[col].describe().to_dict().items():
                d[k] = v
            info_list.append(d)

    Report = pd.DataFrame(info_list)
    return Report


In [2]:
# US_COUNTY_POPULATION[US_COUNTY_POPULATION['Total_Whole'] > 100000]

path = 'DataMerge/Data/C-AnnualCountyPopulation/US_COUNTY_POPULATION.csv'
US_COUNTY_POPULATION = pd.read_csv(path)
US_COUNTY_POPULATION.head()

,GEOID,State,County,Geographic Area Name,Total_Whole,Total_White,Total_Black,Total_AIndA,Total_Asian,Total_WhiteC,Total_BlackC,Total_AIndAC,Total_AsianC
0,1001,Alabama,Autauga,"Autauga, Alabama",55869,42607,11237,266,656,43563,11784,575,950
1,1003,Alabama,Baldwin,"Baldwin, Alabama",223234,195198,19593,1742,2380,199103,21354,3603,3351
2,1005,Alabama,Barbour,"Barbour, Alabama",24686,12134,11906,170,116,12380,12117,297,152
3,1007,Alabama,Bibb,"Bibb, Alabama",22394,17191,4763,103,48,17420,4907,207,97
4,1009,Alabama,Blount,"Blount, Alabama",57826,55412,978,370,185,56167,1276,820,312


In [3]:
STATE_VACCINE = {}

In [4]:
print("Enter the date of collecting data (Today's data in YYYY-MM-DD):")
Today_Date = input()

assert Today_Date == str(datetime.now().date())

print('Hello, ' + Today_Date)


Enter the date of collecting data (Today's data in YYYY-MM-DD):


 2021-04-19


Hello, 2021-04-19


# V. Tableau States

## D. New York (Ask GitHub)


Keep an Eye on 

https://github.com/bertrandmartel/tableau-scraping/issues/9

In [5]:
from tableauscraper import TableauScraper as TS

url = "https://covid19tracker.health.ny.gov/views/Race_Ethnicity_Public/RacebyCounty"

ts = TS()
ts.loads(url)

workbook = ts.getWorkbook()

parameters = workbook.getParameters()
print(parameters)

ts = TS()
ts.loads(url)

# set parameters column / value
workbook = workbook.setParameter('Show Value as', "Number")
# display worksheets
Data = workbook.getWorksheet('Race').data

[{'column': 'Show Value as', 'values': ['Number', 'Percentage'], 'parameterName': '[Parameters].[Parameter 1]'}]


In [6]:
import pandas as pd

L = []

for county, df in Data[Data['Measure Names-alias'] == 'People with at least one vaccine dose '].groupby('County-alias'):
    county, df
    # print(county)
    if county == '%all%':
        continue
    D = {}
    D['County'] = county
    D['Date'] = Today_Date
    for k, row in df.iterrows():
        d = row.to_dict()
        # print(d)
        Race = d['Race-value'] +'_Num'
        Numb = d['Measure Values-alias']
        Numb = int(Numb.replace(',',''))
        D[Race] = Numb

    L.append(D)

NewYork = pd.DataFrame(L)
NewYork

,County,Date,Other_Num,Asian_Num,African American_Num,White_Num
0,Albany,2021-04-19,973.0,9446,9835,102832
1,Allegany,2021-04-19,44.0,123,75,9117
2,Bronx,2021-04-19,8680.0,24901,103991,77257
3,Broome,2021-04-19,599.0,3578,2269,69065
4,Cattaraugus,2021-04-19,463.0,173,183,18627
...,...,...,...,...,...,...
57,Washington,2021-04-19,73.0,131,112,19465
58,Wayne,2021-04-19,172.0,219,593,29910
59,Westchester,2021-04-19,4456.0,28848,35847,287978
60,Wyoming,2021-04-19,31.0,66,50,11492


In [7]:
df

,Race-value,Race-alias,Region-value,Region-alias,County-value,County-alias,Measure Names-value,Measure Names-alias,Measure Values-alias
128,Other,Other,Finger Lakes,Finger Lakes,Yates,Yates,[federated.0nvcif80f24s4c147rz6d1mpve6p (copy)...,People with at least one vaccine dose,12
138,Asian,Asian,Finger Lakes,Finger Lakes,Yates,Yates,[federated.0nvcif80f24s4c147rz6d1mpve6p (copy)...,People with at least one vaccine dose,23
148,African American,African American,Finger Lakes,Finger Lakes,Yates,Yates,[federated.0nvcif80f24s4c147rz6d1mpve6p (copy)...,People with at least one vaccine dose,34
158,White,White,Finger Lakes,Finger Lakes,Yates,Yates,[federated.0nvcif80f24s4c147rz6d1mpve6p (copy)...,People with at least one vaccine dose,"8,086"


In [8]:
# print(len(L))

import numpy as np


def convert_integer(v):
    if type(v) == str:
        v = float(int(v.replace(',', '')))
    return v



state = 'New York' ## 

new_L = []

for idx, row in NewYork.iterrows(): ## 
    d = row.to_dict()
    # print(d)
    new_d = {}
    
    if type(d['County']) != str or d['County'] == 'Total' :
        print(d)
        continue 
        
    # try:
    new_d['State'] = state
    new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
    RenameCounties = {
        # 'Mckean': 'McKean'
        'Mccormick': 'McCormick'
    }
    if new_d['County'] in RenameCounties:
        new_d['County'] = RenameCounties[new_d['County']]

    new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']



    new_d['Numb_White'] = convert_integer(d['White_Num'])
    # new_d['Rate_White'] = convert_percentage(d['White_Rate']) #  if type(d['White_Rate']) == str else d['White_Rate'] 

    new_d['Numb_Black'] = convert_integer(d['African American_Num'])
    # new_d['Rate_Black'] = convert_percentage(d['African American_Rate'])

    new_d['Numb_Asian'] = convert_integer(d['Asian_Num'])
    # new_d['Rate_Asian'] = convert_percentage(d['Asian_Rate'])

    new_d['Date'] = d['Date']

    new_L.append(new_d)

#     except:
#         print(d)

    
Data = pd.DataFrame(new_L)


not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


for Race in ['White', 'Black', 'Asian']:
    Rate = T['Numb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race] = Rate

    Rate = T['Numb_' + Race] / T['Total_' + Race + 'C']
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race + 'C'] = Rate


T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]

print(T.shape)
T.head() # Allogany

(62, 22)


,State,County,Geographic Area Name,Date,Numb_Asian,Numb_Black,Numb_White,Rate_Asian,Rate_AsianC,Rate_Black,...,Rate_WhiteC,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,New York,Albany,"Albany, New York",2021-04-19,9446,9835,102832,0.452286,0.392048,0.228211,...,0.429512,930,2401,20885,24094,43096,48463,231538,239416,305506
1,New York,Allegany,"Allegany, New York",2021-04-19,123,75,9117,0.213172,0.154717,0.112952,...,0.204330,147,324,577,795,664,964,44048,44619,46091
2,New York,Bronx,"Bronx, New York",2021-04-19,24901,103991,77257,0.384952,0.348763,0.168119,...,0.114333,41319,59291,64686,71398,618556,663574,634280,675719,1418207
3,New York,Broome,"Broome, New York",2021-04-19,3578,2269,69065,0.424739,0.351992,0.187273,...,0.408949,519,1680,8424,10165,12116,15644,163473,168884,190488
4,New York,Cattaraugus,"Cattaraugus, New York",2021-04-19,173,183,18627,0.268217,0.184632,0.162956,...,0.260605,2742,3292,645,937,1123,2025,69919,71476,76117


In [9]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['New York']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,62.0,0.171704,0.064905,0.022452,0.127794,0.184903,0.209408,0.409332
1,Rate_White,62.0,0.360685,0.079675,0.121803,0.308196,0.349475,0.416392,0.619115
2,Rate_Parity,62.0,0.188981,0.082928,-0.046316,0.147393,0.188700,0.236617,0.391336


## D. Wisconsin (Done)

New Method: https://github.com/bertrandmartel/tableau-scraping/issues/5

https://www.dhs.wisconsin.gov/covid-19/vaccine-data.htm#day


In [10]:

from tableauscraper import TableauScraper as TS

# url = "https://bi.wisconsin.gov/t/DHS/views/VaccinesAdministeredtoWIResidents/VaccinatedWisconsin-County"
url = 'https://bi.wisconsin.gov/t/DHS/views/VaccinesAdministeredtoWIResidents_16129838459350/VaccinatedWisconsin-County'

ts = TS()
ts.loads(url)

worksheet = ts.getWorksheet("Map")

SelectionDict = {i['column']:i['values'] for i in worksheet.getSelectableItems()}
[i for i in SelectionDict]

['County',
 'SUM(Number With One Dose)',
 'SUM(Number With Two Doses)',
 'SUM(Popcount)',
 'AGG(Number complete for map)',
 'Latitude (generated)',
 'Longitude (generated)',
 'AGG(% complete)',
 'AGG(Calc- Initiation or Full Coverage)',
 'AGG(% at least 1 dose)']

In [11]:
dashboard = worksheet.select("County", "Price County")
df = dashboard.getWorksheet("Race vax/unvax county").data
df

,Race-value,Race-alias,SUM(Initiation or completed count for TT)-alias,AGG(min(1))-value,AGG(min(1))-alias,ATTR(initiation or completed text for TT)-alias,AGG(Geography TT)-alias,AGG(Calc- Initiation or Full Coverage)-alias,Measure Names-alias
0,1,White,4795,1,1,have received at least one dose,Price County,0.37321,min(1)
1,2,Black,12,1,1,have received at least one dose,Price County,0.082192,min(1)
2,4,Asian,32,1,1,have received at least one dose,Price County,0.142222,min(1)
3,3,American Indian,34,1,1,have received at least one dose,Price County,0.257576,min(1)


In [12]:
def process_county(county, df):
    county = county.replace(' County', '')
    D = {}
    D['County'] = county
    for idx, row in df.iterrows():
        d = row.to_dict()
        race = d['Race-alias']
        D['Numb_' + race] = d['SUM(Initiation or completed count for TT)-alias']
        D['Rate_' + race] = d['AGG(Calc- Initiation or Full Coverage)-alias']
    return D

county = "Price County"
df = df

process_county(county, df)

{'County': 'Price',
 'Numb_White': 4795,
 'Rate_White': 0.3732098381070984,
 'Numb_Black': 12,
 'Rate_Black': 0.0821917808219178,
 'Numb_Asian': 32,
 'Rate_Asian': 0.1422222222222222,
 'Numb_American Indian': 34,
 'Rate_American Indian': 0.2575757575757576}

In [13]:


def get_Wisconsin_Raw():
    
    # url = "https://bi.wisconsin.gov/t/DHS/views/VaccinesAdministeredtoWIResidents/VaccinatedWisconsin-County"
    url = 'https://bi.wisconsin.gov/t/DHS/views/VaccinesAdministeredtoWIResidents_16129838459350/VaccinatedWisconsin-County'


    ts = TS()
    ts.loads(url)
    worksheet = ts.getWorksheet("Map")
    SelectionDict = {i['column']:i['values'] for i in worksheet.getSelectableItems()}
    
    L = []

    for county in SelectionDict['County']:
        try:
            dashboard = ts.getWorksheet("Map").select("County", county)
            df = dashboard.getWorksheet("Race vax/unvax county").data
            df_dict = process_county(county, df)
            L.append(df_dict)
            print('Success\t\t', county)


        except:
            print('Fail', county)

    
    Wisconsin = pd.DataFrame(L)
    
    return Wisconsin


Wisconsin = get_Wisconsin_Raw()

Success		 Wood County
Success		 Winnebago County
Success		 Waushara County
Success		 Waupaca County
Success		 Waukesha County
Success		 Washington County
Success		 Washburn County
Success		 Walworth County
Success		 Vilas County
Success		 Vernon County
Success		 Trempealeau County
Success		 Taylor County
Success		 St croix County
Success		 Sheboygan County
Success		 Shawano County
Success		 Sawyer County
Success		 Sauk County
Success		 Rusk County
Success		 Rock County
Success		 Richland County
Success		 Racine County
Success		 Price County
Success		 Portage County
Success		 Polk County
Success		 Pierce County
Success		 Pepin County
Success		 Ozaukee County
Success		 Outagamie County
Success		 Oneida County
Success		 Oconto County
Success		 Monroe County
Success		 Milwaukee County
Success		 Menominee County
Success		 Marquette County
Success		 Marinette County
Success		 Marathon County
Success		 Manitowoc County
Success		 Lincoln County
Success		 Langlade County
Success		 Lafayette Cou

In [14]:

# Wisconsin.to_csv('FinalResult/Raw_Wisconsin.csv')
print(Wisconsin.shape)
Wisconsin.head()

(72, 9)


,County,Numb_White,Rate_White,Numb_Black,Rate_Black,Numb_Asian,Rate_Asian,Numb_American Indian,Rate_American Indian
0,Wood,24826,0.356439,108,0.104550,494,0.328239,107,0.131936
1,Winnebago,56745,0.355686,491,0.093631,1565,0.275674,266,0.183575
2,Waushara,6269,0.267221,20,0.032626,33,0.214286,20,0.092593
3,Waupaca,15518,0.311663,39,0.091335,84,0.260870,74,0.164444
4,Waukesha,161049,0.426769,1974,0.234080,6426,0.382295,521,0.328086


In [15]:
state = 'Wisconsin'


def standardize_Wisconsin(Wisconsin, US_COUNTY_POPULATION):
    


    state = 'Wisconsin' ## 

    new_L = []

    for idx, row in Wisconsin.iterrows(): ## 
        d = row.to_dict()

        new_d = {}


        new_d['State'] = state
        new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
        

        RenameCounties = {
            'Fond Du Lac': 'Fond du Lac',
            'St Croix': 'St. Croix',
        }
        if new_d['County'] in RenameCounties:
            new_d['County'] = RenameCounties[new_d['County']]

        new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']


        new_d['Numb_White'] = d['Numb_White']
        new_d['Rate_White'] = d['Rate_White']

        new_d['Numb_Black'] = d['Numb_Black']
        new_d['Rate_Black'] = d['Rate_Black']

        new_d['Numb_Asian'] = d['Numb_Asian']
        new_d['Rate_Asian'] = d['Rate_Asian']

        new_d['Numb_AIndA'] = d['Numb_American Indian']
        new_d['Rate_AIndA'] = d['Rate_American Indian']

        new_d['Numb_Other'] = np.nan
        new_d['Rate_Other'] = np.nan


        new_d['Numb_NaHis'] = np.nan
        new_d['Rate_NaHis'] = np.nan

        new_d['Numb_Hispa'] = np.nan
        new_d['Rate_Hispa'] = np.nan

        new_d['Date'] = str(datetime.now().date())

        new_L.append(new_d)

    Data = pd.DataFrame(new_L)

    not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
    if len(not_included) > 0:
        print('Not included Counties', not_included)


    T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


    for Race in ['White', 'Black', 'Asian', 'AIndA']:

        Rate = T['Rate_' + Race] 
        T['Rate_' + Race + 'C'] = Rate


    T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]


    return Data, T


    
    
Data, T = standardize_Wisconsin(Wisconsin, US_COUNTY_POPULATION)

# T.to_csv('FinalResult/T_' + state +'.csv', index = False)
print(T.shape)
T.head()


# diagnose_Rate(T)


(72, 31)


,State,County,Geographic Area Name,Date,Numb_AIndA,Numb_Asian,Numb_Black,Numb_Hispa,Numb_NaHis,Numb_Other,...,Rate_WhiteC,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,Wisconsin,Wood,"Wood, Wisconsin",2021-04-19,107,494,108,NaN,NaN,NaN,...,0.356439,726,1036,1448,1660,702,1195,69122,70071,72999
1,Wisconsin,Winnebago,"Winnebago, Wisconsin",2021-04-19,266,1565,491,NaN,NaN,NaN,...,0.355686,1314,2190,5258,6180,4302,5850,157796,160790,171907
2,Wisconsin,Waushara,"Waushara, Wisconsin",2021-04-19,20,33,20,NaN,NaN,NaN,...,0.267221,193,350,133,186,521,644,23264,23568,24443
3,Wisconsin,Waupaca,"Waupaca, Wisconsin",2021-04-19,74,84,39,NaN,NaN,NaN,...,0.311663,398,698,299,442,275,494,49381,49989,50990
4,Wisconsin,Waukesha,"Waukesha, Wisconsin",2021-04-19,521,6426,1974,NaN,NaN,NaN,...,0.426769,1358,2854,15536,18090,7091,9611,373770,379839,404198


In [16]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['New York', 'Wisconsin']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,72.0,0.09874,0.053186,0.000000,0.063957,0.092483,0.122570,0.340437
1,Rate_White,72.0,0.35444,0.063621,0.204664,0.320465,0.345487,0.383666,0.556507
2,Rate_Parity,72.0,0.25570,0.064629,0.102634,0.219181,0.248293,0.285154,0.556507


## D. Ohio (Done)

https://coronavirus.ohio.gov/wps/portal/gov/covid-19/dashboards/covid-19-vaccine/covid-19-vaccination-dashboard

In [17]:
from tableauscraper import TableauScraper as TS

url = "https://public.tableau.com/views/VaccineAdministrationMetricsDashboard/PublicCountyDash"
ts = TS()
ts.loads(url)
dashboard = ts.getWorkbook()

countryDashboard = ts.getWorksheet("New Map").select("county", 'Wyandot')

raceDashboard = dashboard.setParameter('Key Metrics', "Race")

df = raceDashboard.getWorksheet('New Demographics').data
df

,Selected Demographic-value,Selected Demographic-alias,pop_pct_by_demographic-value,pop_pct_by_demographic-alias,SUM(Chosen Dose Status Metric)-value,SUM(Chosen Dose Status Metric)-alias
0,White,White,0.273533,0.273533,5803,5803
1,Unknown,Unknown,%null%,NA,1143,1143
2,Other,Other,%null%,NA,368,368
3,Black or African American,Black or African American,0.16092,0.16092,14,14
4,Asian,Asian,0.222222,0.222222,34,34
5,American Indian or Alaska Native,American Indian Alaska Native,0.169231,0.169231,11,11
6,Native Hawaiian or Other Pacific Islander,Native Hawaiian Pacific Islander,0.75,0.75,6,6
7,Multiracial,Multiracial,0.057377,0.057377,14,14


In [18]:
# from tableauscraper import TableauScraper as TS

# url = "https://public.tableau.com/views/VaccineAdministrationMetricsDashboard/PublicCountyDash"
# ts = TS()
# ts.loads(url)
# dashboard = ts.getWorkbook()

# countryDashboard = ts.getWorksheet("New Map").select("county", 'Williams')

# raceDashboard = dashboard.setParameter('Key Metrics', "Race")

# df = raceDashboard.getWorksheet('New Demographics').data
# df

In [19]:
def process_county(county, df):
    county = county.replace(' County', '')
    D = {}
    D['County'] = county
    for idx, row in df.iterrows():
        d = row.to_dict()
        race = d['Selected Demographic-value']
        D['Numb_' + race] = d['SUM(Chosen Dose Status Metric)-value']
        D['Rate_' + race] = d['pop_pct_by_demographic-value']
    return D

county = 'Williams'
d = process_county(county, df)

sum([v for k, v in d.items() if 'Rate' in k and type(v) == float])
    

1.6332824585440906

In [20]:
from tableauscraper import TableauScraper as TS

url = "https://public.tableau.com/views/VaccineAdministrationMetricsDashboard/PublicCountyDash"
ts = TS()
ts.loads(url)
workbook  = ts.getWorkbook()
worksheet = ts.getWorksheet("New Map")

counties = [
    t["values"] for t in worksheet.getSelectableItems()
    if t["column"] == "county"
][0]


print(counties)



['Wyandot', 'Wood', 'Williams', 'Wayne', 'Washington', 'Warren', 'Vinton', 'Van Wert', 'Union', 'Tuscarawas', 'Trumbull', 'Summit', 'Stark', 'Shelby', 'Seneca', 'Scioto', 'Sandusky', 'Ross', 'Richland', 'Putnam', 'Preble', 'Portage', 'Pike', 'Pickaway', 'Perry', 'Paulding', 'Ottawa', 'Noble', 'Muskingum', 'Morrow', 'Morgan', 'Montgomery', 'Monroe', 'Miami', 'Mercer', 'Meigs', 'Medina', 'Marion', 'Mahoning', 'Madison', 'Lucas', 'Lorain', 'Logan', 'Licking', 'Lawrence', 'Lake', 'Knox', 'Jefferson', 'Jackson', 'Huron', 'Holmes', 'Hocking', 'Highland', 'Henry', 'Harrison', 'Hardin', 'Hancock', 'Hamilton', 'Guernsey', 'Greene', 'Geauga', 'Gallia', 'Fulton', 'Franklin', 'Fayette', 'Fairfield', 'Erie', 'Delaware', 'Defiance', 'Darke', 'Cuyahoga', 'Crawford', 'Coshocton', 'Columbiana', 'Clinton', 'Clermont', 'Clark', 'Champaign', 'Carroll', 'Butler', 'Brown', 'Belmont', 'Auglaize', 'Athens', 'Ashtabula', 'Ashland', 'Allen', 'Adams']


In [21]:
L = []

for county in counties:
    try:
        
        
        url = "https://public.tableau.com/views/VaccineAdministrationMetricsDashboard/PublicCountyDash"
        
        ts = TS()
        ts.loads(url)
        dashboard = ts.getWorkbook()

        countryDashboard = ts.getWorksheet("New Map").select("county", county)

        raceDashboard = dashboard.setParameter('Key Metrics', "Race")

        df = raceDashboard.getWorksheet('New Demographics').data

        df_dict = process_county(county, df)
        L.append(df_dict)
        
        print('Success\t\t', county)
        # time.sleep(0.1)
        
    except:
        print('Fail', county)
    
# len(L)
Ohio = pd.DataFrame(L)
print(Ohio.shape)
Ohio.head()

Success		 Wyandot
Success		 Wood
Success		 Williams
Success		 Wayne
Success		 Washington
Success		 Warren
Success		 Vinton
Success		 Van Wert
Success		 Union
Success		 Tuscarawas
Success		 Trumbull
Success		 Summit
Success		 Stark
Success		 Shelby
Success		 Seneca
Success		 Scioto
Success		 Sandusky
Success		 Ross
Success		 Richland
Success		 Putnam
Success		 Preble
Success		 Portage
Success		 Pike
Success		 Pickaway
Success		 Perry
Success		 Paulding
Success		 Ottawa
Success		 Noble
Success		 Muskingum
Success		 Morrow
Success		 Morgan
Success		 Montgomery
Success		 Monroe
Success		 Miami
Success		 Mercer
Success		 Meigs
Success		 Medina
Success		 Marion
Success		 Mahoning
Success		 Madison
Success		 Lucas
Success		 Lorain
Success		 Logan
Success		 Licking
Success		 Lawrence
Success		 Lake
Success		 Knox
Success		 Jefferson
Success		 Jackson
Success		 Huron
Success		 Holmes
Success		 Hocking
Success		 Highland
Success		 Henry
Success		 Harrison
Success		 Hardin
Success		 Hancock
Succe

,County,Numb_White,Rate_White,Numb_Unknown,Rate_Unknown,Numb_Other,Rate_Other,Numb_Black or African American,Rate_Black or African American,Numb_Asian,Rate_Asian,Numb_American Indian or Alaska Native,Rate_American Indian or Alaska Native,Numb_Native Hawaiian or Other Pacific Islander,Rate_Native Hawaiian or Other Pacific Islander,Numb_Multiracial,Rate_Multiracial
0,Wyandot,5803,0.273533,1143,%null%,368,%null%,14,0.160920,34,0.222222,11.0,0.169231,6.0,0.750000,14,0.057377
1,Wood,48228,0.396107,2140,%null%,3549,%null%,867,0.234071,1302,0.538684,133.0,0.310748,42.0,0.724138,131,0.053360
2,Williams,7834,0.221062,268,%null%,2994,%null%,25,0.052083,51,0.242857,9.0,0.068182,4.0,0.363636,110,0.261283
3,Wayne,29516,0.267896,1436,%null%,1328,%null%,424,0.223865,427,0.338889,40.0,0.163934,13.0,0.260000,124,0.059472
4,Washington,18094,0.314875,1742,%null%,575,%null%,159,0.203065,106,0.257908,23.0,0.136095,9.0,0.642857,12,0.011215


In [22]:

# print(len(L))

import numpy as np

state = 'Ohio' ## 

new_L = []

for idx, row in Ohio.iterrows(): ## 
    d = row.to_dict()
    
    new_d = {}
    
    
    new_d['State'] = state
    new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
    RenameCounties = {
        # 'Mckean': 'McKean'
        'Mccormick': 'McCormick'
    }
    if new_d['County'] in RenameCounties:
        new_d['County'] = RenameCounties[new_d['County']]

    new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']

    new_d['Numb_White'] = d['Numb_White']
    new_d['Rate_White'] = d['Rate_White']
    
    new_d['Numb_Black'] = d['Numb_Black or African American']
    new_d['Rate_Black'] = d['Rate_Black or African American']
    
    new_d['Numb_Asian'] = d['Numb_Asian']
    new_d['Rate_Asian'] = d['Rate_Asian']
    
    new_d['Numb_AIndA'] = d['Numb_American Indian or Alaska Native']
    new_d['Rate_AIndA'] = d['Rate_American Indian or Alaska Native']
    
    new_d['Numb_Other'] = np.nan
    new_d['Rate_Other'] = np.nan
    
    
    new_d['Numb_NaHis'] = np.nan
    new_d['Rate_NaHis'] = np.nan
    
    new_d['Numb_Hispa'] = np.nan
    new_d['Rate_Hispa'] = np.nan
    
    new_d['Date'] = np.nan
    
    new_L.append(new_d)

    
    
Data = pd.DataFrame(new_L)

not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


for Race in ['White', 'Black', 'Asian', 'AIndA']:

    Rate = T['Rate_' + Race] 
    T['Rate_' + Race + 'C'] = Rate


T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]


print(T.shape)
T.head()

(88, 31)


,State,County,Geographic Area Name,Date,Numb_AIndA,Numb_Asian,Numb_Black,Numb_Hispa,Numb_NaHis,Numb_Other,...,Rate_WhiteC,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,Ohio,Wyandot,"Wyandot, Ohio",NaN,11.0,34,14,NaN,NaN,NaN,...,0.273533,65,158,153,190,87,210,21215,21452,21772
1,Ohio,Wood,"Wood, Ohio",NaN,133.0,1302,867,NaN,NaN,NaN,...,0.396107,428,1187,2417,3082,3704,4931,121755,124089,130817
2,Ohio,Williams,"Williams, Ohio",NaN,9.0,51,25,NaN,NaN,NaN,...,0.221062,132,316,210,280,480,678,35438,35837,36692
3,Ohio,Wayne,"Wayne, Ohio",NaN,40.0,427,424,NaN,NaN,NaN,...,0.267896,244,871,1260,1660,1894,3082,110177,112174,115710
4,Ohio,Washington,"Washington, Ohio",NaN,23.0,106,159,NaN,NaN,NaN,...,0.314875,169,655,411,618,783,1278,57464,58470,59911


In [23]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['New York', 'Wisconsin', 'Ohio']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,88.0,0.181406,0.060590,0.022989,0.142208,0.182215,0.219720,0.371453
1,Rate_White,88.0,0.296787,0.065892,0.111968,0.255898,0.281156,0.334601,0.467292
2,Rate_Parity,88.0,0.115380,0.058856,-0.144717,0.085085,0.116810,0.150569,0.241045


## D. South Carolina* (Done)

In [24]:
from tableauscraper import TableauScraper as TS

url = 'https://public.tableau.com/views/COVIDVaccineDashboard/RECIPIENTVIEW'
ts = TS()
ts.loads(url)

ws = ts.getWorksheet("Vaccine Map By SC residents PEOPLE")

ws.data

,Recipient County for maps-value,Recipient County for maps-alias,Recipient Zip Code for maps-alias,Latitude (generated)-value,Latitude (generated)-alias,Longitude (generated)-value,Longitude (generated)-alias,AGG(Count individuals with Suppression )-alias,AGG(Count individuals with Suppression for bar graph)-alias
0,York,York,29945,32.6803,32.680,-80.7941,-80.794,"1,322","1,322"
1,York,York,29944,32.7909,32.791,-81.0332,-81.033,"1,465","1,465"
2,York,York,29943,32.4596,32.460,-81.1515,-81.152,119,119
3,York,York,29941,32.5534,32.553,-80.8144,-80.814,364,364
4,York,York,29940,32.5557,32.556,-80.6994,-80.699,"1,031","1,031"
...,...,...,...,...,...,...,...,...,...
592,Abbeville,Abbeville,0,0,0,0,0,0,0
593,Abbeville,Abbeville,0,0,0,0,0,0,0
594,Abbeville,Abbeville,0,0,0,0,0,0,0
595,Abbeville,Abbeville,0,0,0,0,0,0,0


In [25]:
# show filter columns and values
filters = ws.getFilters()
filters

[{'column': 'Recipient County for maps',
  'values': ['Abbeville',
   'Aiken',
   'Allendale',
   'Anderson',
   'Bamberg',
   'Barnwell',
   'Beaufort',
   'Berkeley',
   'Calhoun',
   'Charleston',
   'Cherokee',
   'Chester',
   'Chesterfield',
   'Clarendon',
   'Colleton',
   'Darlington',
   'Dillon',
   'Dorchester',
   'Edgefield',
   'Fairfield',
   'Florence',
   'Georgetown',
   'Greenville',
   'Greenwood',
   'Hampton',
   'Horry',
   'Jasper',
   'Kershaw',
   'Lancaster',
   'Laurens',
   'Lee',
   'Lexington',
   'Marion',
   'Marlboro',
   'McCormick',
   'nan',
   'Newberry',
   'Oconee',
   'Orangeburg',
   'Pickens',
   'Richland',
   'Saluda',
   'Spartanburg',
   'Sumter',
   'Union',
   'Williamsburg',
   'York'],
  'globalFieldName': '[federated.1ls3et11scyr7y1dl7nmz0wn7p7x].[none:Calculation_1146166120089477132:nk]'}]

In [26]:


def process_county(county, df):
    df = df.replace(-1, 0)
    county = county.replace(' County', '')
    D = {}
    D['County'] = county
    for k, racedf in df.groupby('Assigned Race-value'):
        Numb = racedf['AGG(Count individuals with Suppression )-alias'].sum()
        if 'Asian' in k:
            k = 'Asian'
        # print(k)
        # print(Numb)å
        D['Numb_' + k] = Numb
    return D




def get_SouthCarolina_Raw():
    
    url = 'https://public.tableau.com/views/COVIDVaccineDashboard/RECIPIENTVIEW'
    ts = TS()
    ts.loads(url)

    ws = ts.getWorksheet("Vaccine Map By SC residents PEOPLE")

    # show filter columns and values
    filters = ws.getFilters()
    # print(filters)

    # get all counties values
    counties = next(iter([
        t["values"]
        for t in filters
        if t["column"] == "Recipient County for maps"
    ]))

    # counties

    L = []

    for county in counties:

        try: 
            # print(county)
            wb = ws.setFilter('Recipient County for maps', county)
            countyWs = wb.getWorksheet("Final Age xSex x Race REC")
            # print(countyWs.data)
            df = countyWs.data
            d = process_county(county, df)

            L.append(d)
            print('Success\t\t', county)

        except:
            print('Fail', county)


    SouthCarolina = pd.DataFrame(L)
    return SouthCarolina





In [27]:
county = 'Williamsburg'

wb = ws.setFilter('Recipient County for maps', county)
countyWs = wb.getWorksheet("Final Age xSex x Race REC")

df = countyWs.data

# southcarolina.process_county(county, df)

process_county(county, df)

{'County': 'Williamsburg',
 'Numb_Asian': 18,
 'Numb_Black': 5163,
 'Numb_Other': 1321,
 'Numb_Unknown': 535,
 'Numb_White': 2772}

In [28]:
county = 'York'

wb = ws.setFilter('Recipient County for maps', county)
countyWs = wb.getWorksheet("Final Age xSex x Race REC")

df = countyWs.data

# southcarolina.process_county(county, df)
process_county(county, df)

{'County': 'York',
 'Numb_Asian': 2123,
 'Numb_Black': 9432,
 'Numb_Other': 8263,
 'Numb_Unknown': 5533,
 'Numb_White': 47757}

In [29]:
# SouthCarolina = southcarolina.get_SouthCarolina_Raw()
SouthCarolina = get_SouthCarolina_Raw()
print(SouthCarolina.shape)
SouthCarolina.head()

Success		 Abbeville
Success		 Aiken
Success		 Allendale
Success		 Anderson
Success		 Bamberg
Success		 Barnwell
Success		 Beaufort
Success		 Berkeley
Success		 Calhoun
Success		 Charleston
Success		 Cherokee
Success		 Chester
Success		 Chesterfield
Success		 Clarendon
Success		 Colleton
Success		 Darlington
Success		 Dillon
Success		 Dorchester
Success		 Edgefield
Success		 Fairfield
Success		 Florence
Success		 Georgetown
Success		 Greenville
Success		 Greenwood
Success		 Hampton
Success		 Horry
Success		 Jasper
Success		 Kershaw
Success		 Lancaster
Success		 Laurens
Success		 Lee
Success		 Lexington
Success		 Marion
Success		 Marlboro
Success		 McCormick
Success		 nan
Success		 Newberry
Success		 Oconee
Success		 Orangeburg
Success		 Pickens
Success		 Richland
Success		 Saluda
Success		 Spartanburg
Success		 Sumter
Success		 Union
Success		 Williamsburg
Success		 York
(47, 6)


,County,Numb_Asian,Numb_Black,Numb_Other,Numb_Unknown,Numb_White
0,Abbeville,0,1723,1091,209,4536
1,Aiken,691,7663,5455,2871,30351
2,Allendale,10,1537,212,57,664
3,Anderson,752,5853,6681,4341,39991
4,Bamberg,5,1853,388,185,1518


In [30]:
SouthCarolina = SouthCarolina.replace(0, np.nan)

In [31]:

def standardize_SouthCarolina(SouthCarolina, US_COUNTY_POPULATION):
    
    state = 'South Carolina' ## 

    new_L = []

    for idx, row in SouthCarolina.iterrows(): ## 
        d = row.to_dict()

        new_d = {}


        new_d['State'] = state
        new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
        

        RenameCounties = {
            # 'Mckean': 'McKean'
            'Mccormick': 'McCormick'
        }
        if new_d['County'] in RenameCounties:
            new_d['County'] = RenameCounties[new_d['County']]

        new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']


        new_d['Numb_White'] = d['Numb_White']
        new_d['Rate_White'] = np.nan

        new_d['Numb_Black'] = d['Numb_Black']
        new_d['Rate_Black'] = np.nan

        new_d['Numb_Asian'] = d['Numb_Asian']
        new_d['Rate_Asian'] = np.nan

        new_d['Numb_AIndA'] = np.nan
        new_d['Rate_AIndA'] = np.nan

        new_d['Numb_Other'] = d['Numb_Other']
        new_d['Rate_Other'] = np.nan


        new_d['Numb_NaHis'] = np.nan
        new_d['Rate_NaHis'] = np.nan

        new_d['Numb_Hispa'] = np.nan
        new_d['Rate_Hispa'] = np.nan

        new_d['Date'] = str(datetime.now().date())

        new_L.append(new_d)


    Data = pd.DataFrame(new_L)

    not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
    if len(not_included) > 0:
        print('Not included Counties', not_included)


    T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


    for Race in ['White', 'Black', 'Asian']:
        Rate = T['Numb_' + Race] / T['Total_' + Race]
        Rate = Rate.replace([-np.inf, np.inf], np.nan)
        T['Rate_' + Race] = Rate

        Rate = T['Numb_' + Race] / T['Total_' + Race + 'C']
        Rate = Rate.replace([-np.inf, np.inf], np.nan)
        T['Rate_' + Race + 'C'] = Rate


    T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]
    # print(T.shape)
    # T.head()

    return Data, T


In [32]:
state = 'South Carolina'



Data, T = standardize_SouthCarolina(SouthCarolina, US_COUNTY_POPULATION)
# T.to_csv('FinalResult/T_' + state +'.csv', index = False)
print(T.shape)
T.head()

Not included Counties ['Nan, South Carolina']
(46, 30)


,State,County,Geographic Area Name,Date,Numb_AIndA,Numb_Asian,Numb_Black,Numb_Hispa,Numb_NaHis,Numb_Other,...,Rate_WhiteC,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,South Carolina,Abbeville,"Abbeville, South Carolina",2021-04-19,NaN,NaN,1723,NaN,NaN,1091,...,0.258727,85,196,89,129,6776,7014,17223,17532,24527
1,South Carolina,Aiken,"Aiken, South Carolina",2021-04-19,NaN,691.0,7663,NaN,NaN,5455,...,0.244524,965,2325,2122,2882,43193,45318,120735,124123,170872
2,South Carolina,Allendale,"Allendale, South Carolina",2021-04-19,NaN,10.0,1537,NaN,NaN,212,...,0.298025,36,71,74,82,6329,6378,2167,2228,8688
3,South Carolina,Anderson,"Anderson, South Carolina",2021-04-19,NaN,752.0,5853,NaN,NaN,6681,...,0.240232,689,1590,2111,2683,32960,35359,163136,166468,202558
4,South Carolina,Bamberg,"Bamberg, South Carolina",2021-04-19,NaN,5.0,1853,NaN,NaN,388,...,0.280022,60,123,76,99,8478,8562,5292,5421,14066


In [33]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
# STATE_VACCINE_TMP = {}
# STATE_VACCINE_TMP[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['New York', 'Wisconsin', 'Ohio', 'South Carolina']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,46.0,0.212288,0.036173,0.153407,0.185041,0.206675,0.235848,0.315589
1,Rate_White,46.0,0.272392,0.057986,0.178349,0.228381,0.260844,0.299822,0.498621
2,Rate_Parity,46.0,0.060104,0.049835,-0.018817,0.019961,0.054111,0.084896,0.183032


## D. Oregon* (Done)

In [34]:
from tableauscraper import TableauScraper as TS

url = 'https://public.tableau.com/views/OregonCOVID-19VaccinationTrends/OregonCountyVaccinationTrends'

ts = TS()
ts.loads(url)

In [35]:
worksheet = ts.getWorksheet("County Map Per Capita new")

SelectionDict = {i['column']:i['values'] for i in worksheet.getSelectableItems()}
# [i for i in SelectionDict]



counties = SelectionDict['Recip Address County']


def process_county(county, df):
    county = county.replace(' County', '')
    D = {}
    D['County'] = county
    for idx, row in df.iterrows():
        d = row.to_dict()
        race = d['Demographic Category-value']
        data = d['AGG(People Count Display)-alias']
        data = float(data.replace(',','')) if data!= 'Suppressed' else np.nan
        D['Numb_' + race] = data
    return D

county = 'COOS'

dashboard = worksheet.select('Recip Address County', county)
df = dashboard.getWorksheet('County - Race-Ethnicity').data
d = process_county(county, df)
d

{'County': 'COOS',
 'Numb_Unknown': 1416.0,
 'Numb_Other Race': 717.0,
 'Numb_White': 18302.0,
 'Numb_NHPI': 1330.0,
 'Numb_Hispanic': 585.0,
 'Numb_Black': 98.0,
 'Numb_Asian': 299.0,
 'Numb_AI/AN': 745.0}

In [36]:
L = []

for county in counties:
    try:
        
        url = 'https://public.tableau.com/views/OregonCOVID-19VaccinationTrends/OregonCountyVaccinationTrends'

        ts = TS()
        ts.loads(url)

        worksheet = ts.getWorksheet("County Map Per Capita new")
        
        
        # countryDashboard = ts.getWorksheet("New Map").select("county", county)

        # raceDashboard = dashboard.setParameter('Key Metrics', "Race")

        dashboard = worksheet.select('Recip Address County', county)
        df = dashboard.getWorksheet('County - Race-Ethnicity').data

        df_dict = process_county(county, df)
        L.append(df_dict)
        
        print('Success\t\t', county)
        # time.sleep(0.1)
        
    except:
        print('Fail', county)
    
# len(L)
Oregon = pd.DataFrame(L)
print(Oregon.shape)
Oregon.head()

Success		 YAMHILL
Success		 WHEELER
Success		 WASHINGTON
Success		 WASCO
Success		 WALLOWA
Success		 UNION
Success		 UMATILLA
Success		 TILLAMOOK
Success		 SHERMAN
Success		 POLK
Success		 MULTNOMAH
Success		 MORROW
Success		 MARION
Success		 MALHEUR
Success		 LINN
Success		 LINCOLN
Success		 LANE
Success		 LAKE
Success		 KLAMATH
Success		 JOSEPHINE
Success		 JEFFERSON
Success		 JACKSON
Success		 HOOD RIVER
Success		 HARNEY
Success		 GRANT
Success		 GILLIAM
Success		 DOUGLAS
Success		 DESCHUTES
Success		 CURRY
Success		 CROOK
Success		 COOS
Success		 COLUMBIA
Success		 CLATSOP
Success		 CLACKAMAS
Success		 BENTON
Success		 BAKER
(36, 9)


,County,Numb_Unknown,Numb_Other Race,Numb_White,Numb_NHPI,Numb_Hispanic,Numb_Black,Numb_Asian,Numb_AI/AN
0,YAMHILL,4398.0,1873.0,25395.0,145.0,3727.0,169.0,520.0,919.0
1,WHEELER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,WASHINGTON,16332.0,19614.0,140124.0,2570.0,20891.0,3869.0,23993.0,2968.0
3,WASCO,1217.0,155.0,6788.0,36.0,1173.0,33.0,104.0,171.0
4,WALLOWA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
# print(len(L))

import numpy as np

state = 'Oregon' ## 
new_L = []


for idx, row in Oregon.iterrows(): ## 
    d = row.to_dict()
    
    new_d = {}
    
    new_d['State'] = state
    new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
    new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']
    
    new_d['Numb_White'] = d['Numb_White'] if d['Numb_White']!= '-' else np.nan
    new_d['Rate_White'] = np.nan
    
    new_d['Numb_Black'] = d['Numb_Black'] if d['Numb_Black']!= '-' else np.nan
    new_d['Rate_Black'] = np.nan
    
    new_d['Numb_Asian'] = d['Numb_Asian'] if d['Numb_Asian']!= '-' else np.nan
    new_d['Rate_Asian'] = np.nan
    
    new_d['Date'] = np.nan
    
    new_L.append(new_d)
    
    
Data = pd.DataFrame(new_L)
not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


for Race in ['White', 'Black', 'Asian']:
    Rate = T['Numb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race] = Rate

    Rate = T['Numb_' + Race] / T['Total_' + Race + 'C']
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race + 'C'] = Rate


T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]
print(T.shape)
T.head()

(36, 22)


,State,County,Geographic Area Name,Date,Numb_Asian,Numb_Black,Numb_White,Rate_Asian,Rate_AsianC,Rate_Black,...,Rate_WhiteC,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,Oregon,Yamhill,"Yamhill, Oregon",NaN,520.0,169.0,25395.0,0.240185,0.153937,0.131313,...,0.251471,2148,3729,2165,3378,1287,2014,97848,100986,107100
1,Oregon,Wheeler,"Wheeler, Oregon",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,34,57,14,30,3,18,1224,1276,1332
2,Oregon,Washington,"Washington, Oregon",NaN,23993.0,3869.0,140124.0,0.340727,0.281873,0.254640,...,0.277840,6827,13719,70417,85120,15194,22100,479049,504333,601592
3,Oregon,Wasco,"Wasco, Oregon",NaN,104.0,33.0,6788.0,0.346667,0.221277,0.153488,...,0.272217,1003,1407,300,470,215,352,24271,24936,26682
4,Oregon,Wallowa,"Wallowa, Oregon",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,68,156,36,68,33,80,6890,7051,7208


In [38]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['New York', 'Wisconsin', 'Ohio', 'South Carolina', 'Oregon']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,23.0,0.243613,0.076807,0.086639,0.189910,0.238095,0.294205,0.385439
1,Rate_White,23.0,0.291034,0.061294,0.145512,0.253347,0.279675,0.338484,0.421104
2,Rate_Parity,23.0,0.047421,0.052244,-0.035555,0.016166,0.047121,0.066982,0.181631


# All in One

## Read Previous Vaccine Data

In [36]:
# import os
# path = ''

# print('Input Historical STATE_VACCINE')
# print('For example: RaceVaccineBackup/STATE_VACCINE_2021-03-23.p')

# L = []
# while True:
    
#     path = input()
    
#     if os.path.isfile(path):
#         print('Path Exists!')
#         L.append(path)
#     elif path != '':
#         print('Wrong Path')
#     else:
#         break

# print('End')

In [37]:
# import pickle

# Historical_Path = {i.split('_')[-1].replace('.p', ''): i for i in L }

# Historical_Data = {}
# for date, path in Historical_Path.items():
#     with open(path, 'rb' ) as f:
#         Historical_Data[date] = pickle.load(f)
    
    
# [i for i in Historical_Data]

## Show Parity for Each State

In [38]:
# ######################################

# # Today_Date = str(datetime.now().date())
# print(Today_Date)
# ######################################


# for state in STATE_VACCINE:
    
#     print(state)
#     cols = ['name',  'date', 'count', 'mean', 'min','50%', 'max']
    
    
    
#     for date, Old_STATE_VACCINE in Historical_Data.items():
#         x = diagnose_Rate(Old_STATE_VACCINE[state], state = state, date = date)
#         print(x[cols])
    
#     x = diagnose_Rate(STATE_VACCINE[state], state = state, date = Today_Date)
    
#     print(x[cols])
#     print('\n')
    
    
# # pd.concat(L).to_csv('compare.csv')

##  Show Some Specials Cases

In [39]:
# # state = 'Texas'
# DF = pd.concat([v for k,v in STATE_VACCINE.items()]) 

# data_cols = ['State', 'County', 'Geographic Area Name',
#              'Total_Black', 'Total_White',
#              'Numb_Black', 'Numb_White','Rate_Black','Rate_White', 'Total_Whole', 'Rate_Parity']

# DF[DF['Rate_Black'] > 0.3][data_cols]

In [49]:
[i for i in STATE_VACCINE]

['Wisconsin', 'Ohio', 'South Carolina', 'Oregon', 'New York']

## Save Data

In [39]:

import pickle 

path = 'CountyVaccine/Workplace/STATE_VACCINE_' + Today_Date + '_Tableau.p'

while True:
    print('The data will be saved here, do you agree?' )
    print(path)
    print('If yes, please enter: Yes')
    x = input()
    if x =='Yes':
        break


with open(path, 'wb') as f:
    pickle.dump(STATE_VACCINE, f)
    
    
print('Successfully Saved!')

The data will be saved here, do you agree?
CountyVaccine/Workplace/STATE_VACCINE_2021-04-19_Tableau.p
If yes, please enter: Yes


 Yes


Successfully Saved!
